# HuggingFace meets `bitsandbytes` for lighter models on GPU for inference

 <center>
 <img src="https://s3.amazonaws.com/moonup/production/uploads/1659861207959-62441d1d9fdefb55a0b7d12c.png">
 </center>


You can run your own 8-bit model on any HuggingFace 🤗 model with just few lines of code. Install the dependencies below first!


In [ ]:
!pip install --quiet bitsandbytes
!pip install --quiet git+https://github.com/huggingface/transformers.git # Install latest version of transformers
!pip install --quiet accelerate

## Hardware requirements 🔨

To run properly this feature you need to have GPU that supports 8-bit operation modules. Currently, Turing and Ampere GPUs (RTX20s, RTX30s, A40-A100, T4+) are supported, which means on colab we need to use a T4 GPU for this feature. You can check that using this code snippet and make sure you are using a supported GPU

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Apr  5 17:29:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Here we are using a `Tesla T4` GPU that should support 8-bit tensor cores! We are good to go 🚀

## Utility variables & functions 🧰

In [12]:
text = "Automation helps the business to operate efficiently and will reduce the workload on employees who will be required to monitor the process and suggest further changes if required."

def generate_from_model(model, tokenizer):
  encoded_input = tokenizer(text, return_tensors='pt')
  output_sequences = model.generate(max_length=200,min_length=100,temperature=0.7,input_ids=encoded_input['input_ids'].cuda())
  return tokenizer.decode(output_sequences[0], skip_special_tokens=True)

## Use 8bit models and `pipeline` 🤗

You can use 8bit quantized models together with `pipeline` as follows:

In [ ]:
from transformers import pipeline

pipe = pipeline(model=name, model_kwargs= {"device_map": "auto", "load_in_8bit": True}, max_new_tokens=max_new_tokens)

Let's check the output!

In [ ]:
pipe(text)

[{'generated_text': 'Hello my name is John and I am a student at the University of the West of England. I am currently studying for'}]

## Use 8bit models and `.generate` 📖

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_8bit = AutoModelForCausalLM.from_pretrained('EleutherAI/gpt-neo-2.7B', device_map="auto", load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-2.7B')

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 4>:4                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/models/auto/auto_factory.py:471 in           │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   468 │   │   │   )                                                                              │
│   469 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   470 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│ ❱ 471 │   │   │   return model_class.from_pretrained(                                            │
│   472 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   473 │   │   │   )                                                                              │
│   474 │   │   raise ValueError(                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:2394 in from_pretrained    │
│                                                                                                  │
│   2391 │   │   │   │   │   │   "_raise_exceptions_for_missing_entries": False,                   │
│   2392 │   │   │   │   │   │   "_commit_hash": commit_hash,                                      │
│   2393 │   │   │   │   │   }                                                                     │
│ ❱ 2394 │   │   │   │   │   resolved_archive_file = cached_file(pretrained_model_name_or_path, f  │
│   2395 │   │   │   │   │                                                                         │
│   2396 │   │   │   │   │   # Since we set _raise_exceptions_for_missing_entries=False, we don't  │
│   2397 │   │   │   │   │   # result when internet is up, the repo and revision exist, but the f  │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/utils/hub.py:409 in cached_file              │
│                                                                                                  │
│    406 │   user_agent = http_user_agent(user_agent)                                              │
│    407 │   try:                                                                                  │
│    408 │   │   # Load from URL or cache if already cached                                        │
│ ❱  409 │   │   resolved_file = hf_hub_download(                                                  │
│    410 │   │   │   path_or_repo_id,                                                              │
│    411 │   │   │   filename,                                                                     │
│    412 │   │   │   subfolder=None if len(subfolder) == 0 else subfolder,                         │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/huggingface_hub/utils/_validators.py:120 in _inner_fn     │
│                                                                                                  │
│   117 │   │   if check_use_auth_token:                                                           │
│   118 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   119 │   │                                                                                      │
│ ❱ 120 │   │   return fn(*args, **kwargs)                   

In [13]:
generate_from_model(model_8bit, tokenizer)

'Automation helps the business to operate efficiently and will reduce the workload on employees who will be required to monitor the process and suggest further changes if required. Automation also helps to reduce the risk of human error and improve the quality of the product. Automation also helps to reduce the cost of production and improve the efficiency of the business.\nAutomation is a process of automating the business processes. Automation is a process of automating the business processes. Automation is a process of automating the business processes. Automation is a process of automating the business processes. Automation is a process of automating the business processes. Automation is a process of automating the business processes. Automation is a process of automating the business processes. Automation is a process of automating the business processes. Automation is a process of automating the business processes. Automation is a process of automating the business processes. Aut

Let's compare the qualitative results between our quantized model and the original model

In [ ]:
model_native = AutoModelForCausalLM.from_pretrained(name, device_map="auto", torch_dtype="auto")
generate_from_model(model_native, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


'Hello my name is John and I am a student at the University of the West Indies. I am'

## Memory footprint comparison 🪶

In [ ]:
mem_fp16 = model_native.get_memory_footprint()
mem_int8 = model_8bit.get_memory_footprint()
print("Memory footprint int8 model: {} | Memory footprint fp16 model: {} | Relative difference: {}".format(mem_int8, mem_fp16, mem_fp16/mem_int8))

Memory footprint int8 model: 3645818880 | Memory footprint fp16 model: 6005114880 | Relative difference: 1.6471237539918604


We saved 1.65x memory for a 3-billion parameters models! Note that internally we replace all the linear layers by the ones implemented in `bitsandbytes`. By scaling up the model the number of linear layers will increase therefore the impact of saving memory on those layers will be huge for very large models. For example quantizing BLOOM-176 (176 Billion parameter model) gives a gain of 1.96x memory footprint which can save a lot of compute power in practice.

## Hyper-parameter tuning 📠


**Warning:** you may want to run these cells separately from previous cells to avoid Out Of Memory (OOM) issues.

You can play with the parameter `int8_threshold` and see its impact in the results of your model. You can directly specify this parameter when loading your model through `.from_pretrained` method. By default we set this parameter to be `6.0` as described in the paper.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_8bit_thresh_4 = AutoModelForCausalLM.from_pretrained(name, device_map="auto", load_in_8bit=True, load_in_8bit_threshold=4.0)
model_8bit_thresh_2 = AutoModelForCausalLM.from_pretrained(name, device_map="auto", load_in_8bit=True, load_in_8bit_threshold=2.0)
tokenizer = AutoTokenizer.from_pretrained(name)

In [ ]:
generate_from_model(model_8bit_thresh_4, tokenizer)

'Hello my name is John and I am a student at the University of the West Indies. I am'

In [ ]:
generate_from_model(model_8bit_thresh_2, tokenizer)

'Hello my name is John and I am a newbie to the forum. I have a question about'

As you can see the generations can slightly vary by using different thresholds. This is because manipulating 8-bit parameters leads to easier perturbations by small changes! Lowering the threshold means also less parameters in fp16 so breaking down the threshold to `0` leads to a full model in `int8`. 